# Training on online dataset
This file is the Google Colab Jupyter notebook used for training our U-Net model
It does not work on a local host, only in Google Colab with the correct files in the google drive
author: Pinglei He, Igor Augusto Oliveira

## Mounting google drive

In [1]:
# mount google drive to colab
from google.colab import drive
drive.mount('/content/drive')

# list content of the directory to ensure the mounting is successful
!ls ./drive/MyDrive/Medical-Noise-Cancellation

# add project root to python module paths
import sys
sys.path.insert(0, "/content/drive/MyDrive/Medical-Noise-Cancellation")

Mounted at /content/drive
archives		 model		unet-training.ipynb
bm3d			 model_gpu	unet-training-online.ipynb
data			 README.md	weights
data-augmentation.ipynb  result_images	weights_first_run
evaluation		 runs


## Hardware information

In [2]:
# colab GPU information
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# determine whether the current is high-RAM for choosing the right batch size
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('Using a high-RAM runtime!')

Tue Apr 26 13:09:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Module imports

In [3]:
import matplotlib.pyplot as plt # for plotting
from torch.utils.data import DataLoader
from torch.nn import MSELoss # mean square error loss
from torch import Tensor
from torch.optim.lr_scheduler import OneCycleLR # learning rate decay
from torch.optim import AdamW
from tqdm import tqdm # progress bar
from torch.utils.tensorboard import SummaryWriter # for saving metrics and losses
import torch
import pickle # for saving model state
import numpy as np
from PIL import Image

In [4]:
# custom modules fitted for batch training with cuda / GPU
from model_gpu import (StentOnlineDataset,
                        StentDataset,
                        UNet) # our PyTorch U-Net model

## Parameters

In [5]:
# root directory of the project
root_dir = "drive/MyDrive/Medical-Noise-Cancellation/" # google drive

# parameters used in training
n_images = 10005 # number of images
batch_size = 5 # training set batch size ie. number of images per iteration
val_batch_size = 1 # validation set batch size
lr = 0.0002 # initial learning rate

# learning rate decay parameters
# scheduler_patience = 80 # set number of validations with no improvement after which the scheduler would decrease the lr
# lr_schedule_factor = 0.1 # new_lr = lr * schedule_factor when learning rate is decayed

# parameters about previously trained / cached weights
use_cached_weights = False # use models weights that we cached at checkpoint
path_weights = root_dir + "weights/unet-model_7000.pkl" # path to cached weights
previous_iter_count = 7000 # number of iterations we trained the model for before launching this notebook if use_cached_weights


## Initialization

In [6]:
# clear GPU memory if cuda used
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [7]:
# instantiate the model
model = None
if use_cached_weights:
    # load pretrained model with pickle
    with open(path_weights, "rb") as f:
        model = pickle.load(f)
else:
    # create new model
    model = UNet(in_channels=1, out_channels=1)
    # use double precision
    # model.double()

# print information on GPU availability of current runtime
if torch.cuda.is_available():
    model = model.cuda()
    if not next(model.parameters()).is_cuda:
      print("There is a CUDA device available, but the model was not successfuly sent to it.")
    else:
      print("Model successfully loaded to cuda.")

Model successfully loaded to cuda.


In [8]:
# custom image dataset class with real-time augmentation
dataset = StentOnlineDataset(n_images=n_images, base_image_path=root_dir + "data/dataset/base_png")
# pytorch dataset object
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# validation set
val_dataset = StentDataset(input_path=root_dir + "data/dataset/val_noisy_png",
                           target_path=root_dir + "data/dataset/val_base_png")
val_data_loader = DataLoader(val_dataset, batch_size=val_batch_size, shuffle=False)

In [9]:
# mean square error for avoiding the model to over-whiten the output image when the stent sizes are small
criterion = MSELoss()
# optimizer = Adam(model.parameters(), lr=lr)
# scheduler = ReduceLROnPlateau(optimizer, 
#                               mode='min', 
#                               patience=scheduler_patience, 
#                               verbose=True, 
#                               factor=lr_schedule_factor)
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = OneCycleLR(optimizer, max_lr=lr, total_steps=int(n_images/batch_size))

## Model Summary

In [10]:
print(model)

UNet(
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_down_0): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
  (conv_down_1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
  (conv_down_2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
  (conv_down_3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
  (conv_down_4): Sequential(
    (0): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(

## Training

In [11]:
# save the losses for analysis
# writer = SummaryWriter(log_dir=root_dir + "runs/2022_04_25_b")
train_losses = []
val_losses = []
lrs = []

In [12]:
# training
for i, data in enumerate(tqdm(data_loader)):
    # get the inputs
    inputs, targets = data
    # send to GPU
    if torch.cuda.is_available():
      inputs = inputs.cuda()
      targets = targets.cuda()
    # zero the parameter gradients
    optimizer.zero_grad()
    # forward + backward + optimize
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    scheduler.step()

    # record the losss
    # writer.add_scalar('Loss/train', loss.item(), i)
    # writer.add_scalar("Log loss/train", np.log(loss.item()), i)
    train_losses.append(loss.item())
    lrs.append(optimizer.param_groups[0]["lr"])

    # print metric on validation set every 10 iterations
    if i % 10 == 0:
      running_loss = 0.0
      for vdata in val_data_loader:
        vinputs, vtargets = vdata
        if torch.cuda.is_available():
          vinputs = vinputs.cuda()
          vtargets = vtargets.cuda()
        voutputs = model(vinputs)
        vloss = criterion(voutputs, vtargets)
        running_loss += vloss.item()
      
      avg_vloss = running_loss / 3
      print(f"Validation loss: {avg_vloss}")
      # writer.add_scalar("Loss/validation", avg_vloss, i)
      # writer.add_scalar("Log loss/validation", np.log(avg_vloss), i)
      # writer.add_scalar("Log10 lr", np.log10(optimizer.param_groups[0]['lr']), i)
      val_losses.append(avg_vloss)


      # learning rate decays when the validation loss plateaus
      

    # checkpoint
    if i % 500 == 0:
      torch.save(model.state_dict(), root_dir + f"weights/iter_{i:05d}.pt")
      # with open(root_dir + "weights/scheduler.pickle", "wb") as f:
      #   pickle.dump(scheduler, f)
      with open(root_dir + "weights/losses.pickle", "wb") as f:
        pickle.dump({"train_losses": train_losses, "val_losses": val_losses, "lrs": lrs}, f)


    # show image
    if i < 20 or i < 200 and i % 5 == 0 or i < 1000 and i % 50 == 0 or i % 200 == 0:
        # if using cuda, copy input tensor and output tensor to CPU for printing
        if torch.cuda.is_available():
          inputs = Tensor.cpu(inputs)
          outputs = Tensor.cpu(outputs)
          targets = Tensor.cpu(targets)
        print(f"Step [{i + 1}/{len(data_loader)}], Loss:{loss.item()}, Lr: {optimizer.param_groups[0]['lr']}")
        # print("Original image:")
        plt.figure()
        plt.imshow(inputs.detach().numpy()[0, 0, 94:-94, 94:-94], cmap="gray")
        plt.savefig(root_dir + f"result_images/iter_{i:05d}_input.png", bbox_inches='tight')
        # plt.show()

        # print("Predicted image:")
        plt.figure()
        plt.imshow(outputs.detach().numpy()[0, 0], cmap="gray")
        plt.savefig(root_dir + f"result_images/iter_{i:05d}_prediction.png", bbox_inches='tight')
        # plt.show()

        # print("Ground truth:")
        plt.figure()
        plt.imshow(targets.detach().numpy()[0, 0], cmap="gray")
        plt.savefig(root_dir + f"result_images/iter_{i:05d}_target.png", bbox_inches='tight')
        # plt.show()

        plt.close("all")

  0%|          | 0/2001 [00:00<?, ?it/s]

Validation loss: 1.0520263691603413
Step [1/2001], Loss:1.0531486072013352, Lr: 8.001319020158374e-06


  0%|          | 1/2001 [00:20<11:31:42, 20.75s/it]

Step [2/2001], Loss:1.037221399809771, Lr: 8.005276044387356e-06


  0%|          | 2/2001 [00:29<7:36:46, 13.71s/it] 

...

100%|█████████▉| 1991/2001 [3:39:15<01:10,  7.03s/it]

Validation loss: 0.0004070681964432429


100%|█████████▉| 2000/2001 [3:40:11<00:06,  6.32s/it]

Validation loss: 0.00040706759145995507
Step [2001/2001], Loss:0.0003346696421933305, Lr: 1.0515229240213496e-09


100%|██████████| 2001/2001 [3:40:26<00:00,  6.61s/it]


## Visualisation des pertes

In [ ]:
iter_count_train = np.linspace(1, 2001, 2001) # training loss and lr are recorded every iteration
iter_count_val = np.linspace(1, 2001, 201) # validation loss is recorded once per 10 iterations

plt.figure()
plt.title("Learning rate variation")
plt.plot(iter_count_train, lrs, label="learning rate") # plot learning rate variation
plt.xlabel("Iteraiton")
plt.ylabel("Learning rate")
plt.legend()
plt.show()

plt.figure()
plt.title("Loss function variation")
plt.plot(iter_count_train, train_losses, label="batch training loss") # loss of the current batch of training set

plt.plot(iter_count_val, val_losses, label="validation loss") # loss on validation set
plt.legend()
plt.xlabel("Iteration")
plt.ylabel("MSE Loss")
plt.show()

plt.figure()
plt.title("Loss function variation (logarithmic scale)")
plt.plot(iter_count_train, train_losses, label="batch training loss") # loss of the current batch of training set
plt.plot(iter_count_val, val_losses, label="validation loss") # loss on validation set
plt.legend()
plt.yscale("log")
plt.xlabel("Iteration")
plt.ylabel("log10 MSE Loss")
plt.show()
